In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import random

from time import time

from scipy.special import comb, loggamma, lambertw
from scipy.stats import multinomial, expon

from silence_tensorflow import silence_tensorflow
silence_tensorflow()
import tensorflow as tf
import tensorflow_probability as tfp

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config = config)

import os, shutil
import json
import subprocess

from net_model import *
from custom_model import *
from mps_models import *

import mps
import pwexp

E0000 00:00:1741902446.277233   94009 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741902446.282783   94009 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1741902448.339840   94009 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1487 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 6GB Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
/home/natan/.pyenv/versions/3.10.16/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_train = pd.read_csv("OASIS - Small Dataset/trainSet.csv")
df_train = df_train.iloc[:,1:]

df_test = pd.read_csv("OASIS - Small Dataset/testSet.csv")
df_test = df_test.iloc[:,1:]

df = pd.concat([df_train, df_test])

print("Dimensão dos dados de treino: {}".format(df_train.shape))
print("Dimensão dos dados de teste: {}".format(df_test.shape))
print("Dimensão dos dados completos: {}".format(df.shape))

mri = np.load("OASIS - Small Dataset/mri.npz")

mriTrain = (mri["train_images"] / np.max(mri["train_images"]))
# mriTrain = mri["train_images"]
# Remove uma observação do conjunto, que contava com dados de especificação faltantes
mriTrain = np.delete(mriTrain, 230, axis=0)

mriTest = (mri["test_images"] / np.max(mri["test_images"]))
# mriTest = mri["test_images"]

image_dim = mriTrain[0].shape
print("Dimensão das imagens: {}".format(image_dim))

x_train = mriTrain
t_train = df_train.time.to_numpy()
delta_train = df_train.delta.to_numpy()
m_train = df_train.delta.to_numpy()
x_test = mriTest
t_test = df_test.time.to_numpy()
delta_test = df_test.delta.to_numpy()
m_test = df_test.delta.to_numpy()

Dimensão dos dados de treino: (280, 23)
Dimensão dos dados de teste: (72, 23)
Dimensão dos dados completos: (352, 23)
Dimensão das imagens: (160, 200, 1)


# Generating bootstrap sample